<center>
<img src="../../img/ods_stickers.jpg">
## Открытый курс по машинному обучению
Авторы материала: программист-исследователь Mail.ru Group, старший преподаватель Факультета Компьютерных Наук ВШЭ Юрий Кашницкий и Data Scientist в Segmento Екатерина Демидова. Материал распространяется на условиях лицензии [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/). Можно использовать в любых целях (редактировать, поправлять и брать за основу), кроме коммерческих, но с обязательным упоминанием автора материала.

# <center>Тема 2. Углубленные приемы pandas, collections, numpy</center>
## <center>Часть 2. Углубленные приемы pandas

In [1]:
import glob
import random
import string

import numpy as np
import pandas as pd

### Индексация

In [2]:
df = pd.DataFrame({'a':[1,2,3,4,5], 
                   'b':[10,20,30,40,50],
                   'c':[100,200,300,400,500]})

In [3]:
df

,a,b,c
0,1,10,100
1,2,20,200
2,3,30,300
3,4,40,400
4,5,50,500


In [4]:
df.iloc[1:4, 1:2]

,b
1,20
2,30
3,40


In [5]:
df.loc[3:4, 'a':'c']

,a,b,c
3,4,40,400
4,5,50,500


In [6]:
df[['a', 'b']][3:]

,a,b
3,4,40
4,5,50


Неправильно

In [7]:
pd.set_option('mode.chained_assignment','warn')
df[df['a']<=3]['b'] = (df[df['a']<=3]['b'])/10

/Users/i.marzharov/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


Правильно

In [8]:
df.loc[df['a'] <= 3, 'b'] = df.loc[df['a'] <= 3, 'b'] / 10
df

,a,b,c
0,1,1.0,100
1,2,2.0,200
2,3,3.0,300
3,4,40.0,400
4,5,50.0,500


### inplace

In [9]:
df1 = pd.DataFrame(np.random.rand(100, 3))
df1[df1 < 0.5] = np.nan
df2 = pd.DataFrame(np.random.rand(100, 3))
df2[df2 < 0.5] = np.nan

In [10]:
%%timeit
df = (df1-df2).dropna()
df = df.rename(columns={col:'d{}'.format(col) for col in df.columns})

2.44 ms ± 95.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [11]:
%%timeit
df = (df1-df2).dropna()
df.rename(columns={col:'d{}'.format(col) for col in df.columns}, inplace=True)

19.2 ms ± 1.64 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [12]:
new_v = df.dropna(inplace=True)#.rename().sum()

### Загрузка файлов

In [13]:
%%timeit
files = glob.glob('logs/*.csv')

columns = ['1','2','3','4','5','6','7','8','9','10']
logs = pd.DataFrame(columns=columns)

for fp in files:
    day_data = pd.read_csv(fp, usecols=columns)
    logs.append(day_data)

658 ms ± 122 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [14]:
%%timeit
files = glob.glob('logs/*.csv')
columns = ['1','2','3','4','5','6','7','8','9','10']
logs_dfs = [pd.read_csv(fp, usecols=columns) for fp in files]
logs = pd.concat(logs_dfs)

420 ms ± 19.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### apply

In [15]:
def create_frame(n, n_groups):
    stamps = pd.date_range('20010101', periods=n, freq='ms')
    random.shuffle(stamps.values)    
    return pd.DataFrame({'name': np.random.randint(0,n_groups,size=n),
                         'stamp': stamps,
                         'value': np.random.randint(0,n,size=n),
                         'value2': np.random.randn(n)})


df = create_frame(1000000,10000)

def f_apply(df):
    return df.groupby('name').value2.apply(lambda x: (x-x.mean())/x.std())

def f_unwrap(df):
    g = df.groupby('name').value2
    v = df.value2
    return (v-g.transform(np.mean))/g.transform(np.std)

In [16]:
df.head()

,name,stamp,value,value2
0,4850,2001-01-01 00:16:32.627,118011,0.122250
1,4434,2001-01-01 00:11:34.112,754006,-1.380408
2,1792,2001-01-01 00:03:31.623,402014,-0.517809
3,12,2001-01-01 00:14:49.378,315222,0.123394
4,622,2001-01-01 00:00:58.515,88042,-0.850993


In [17]:
%timeit f_apply(df)

6.02 s ± 140 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [18]:
%timeit f_unwrap(df)

60.5 ms ± 1.49 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### category

In [19]:
s = pd.Series(np.random.choice(list(string.ascii_letters), 100000))
s.head()

0    K
1    E
2    D
3    N
4    S
dtype: object

In [20]:
'{:0.2f} KB'.format(s.memory_usage(index=False) / 1000)

'800.00 KB'

In [21]:
c = s.astype('category')
'{:0.2f} KB'.format(c.memory_usage(index=False) / 1000)

'102.98 KB'

In [22]:
df = pd.read_csv('game_logs.csv')

/Users/i.marzharov/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (12,13,14,15,19,20,81,83,85,87,93,94,95,96,97,98,99,100,105,106,108,109,111,112,114,115,117,118,120,121,123,124,126,127,129,130,132,133,135,136,138,139,141,142,144,145,147,148,150,151,153,154,156,157,160) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [23]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 171907 entries, 0 to 171906
Columns: 161 entries, date to acquisition_info
dtypes: float64(77), int64(6), object(78)
memory usage: 861.6 MB


In [24]:
object_columns = []
for col in df.columns:
    if df[col].dtype == 'object':
        num_unique_values = len(df[col].unique())
        num_total_values = len(df[col])
        if num_unique_values / num_total_values < 0.5:
            object_columns.append(col)
            df.loc[:,col] = df[col].astype('category')

In [25]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 171907 entries, 0 to 171906
Columns: 161 entries, date to acquisition_info
dtypes: category(78), float64(77), int64(6)
memory usage: 160.5 MB


In [26]:
for col in df.columns:
    if df[col].dtype == 'int64':
        df.loc[:,col] = df[col].astype('int32')

In [27]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 171907 entries, 0 to 171906
Columns: 161 entries, date to acquisition_info
dtypes: category(78), float64(77), int32(6)
memory usage: 156.6 MB


In [28]:
for col in df.columns:
    if df[col].dtype == 'float64':
        df.loc[:,col] = df[col].astype('float32')

In [29]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 171907 entries, 0 to 171906
Columns: 161 entries, date to acquisition_info
dtypes: category(78), float32(77), int32(6)
memory usage: 106.1 MB


In [30]:
column_types = {col: 'category' for col in object_columns}
df = pd.read_csv('game_logs.csv', dtype=column_types)

In [31]:
df.head()

,date,number_of_game,day_of_week,v_name,v_league,v_game_number,h_name,h_league,h_game_number,v_score,...,h_player_7_name,h_player_7_def_pos,h_player_8_id,h_player_8_name,h_player_8_def_pos,h_player_9_id,h_player_9_name,h_player_9_def_pos,additional_info,acquisition_info
0,18710504,0,Thu,CL1,na,1,FW1,na,1,0,...,Ed Mincher,7.0,mcdej101,James McDermott,8.0,kellb105,Bill Kelly,9.0,NaN,Y
1,18710505,0,Fri,BS1,na,1,WS3,na,1,20,...,Asa Brainard,1.0,burrh101,Henry Burroughs,9.0,berth101,Henry Berthrong,8.0,HTBF,Y
2,18710506,0,Sat,CL1,na,2,RC1,na,1,12,...,Pony Sager,6.0,birdg101,George Bird,7.0,stirg101,Gat Stires,9.0,NaN,Y
3,18710508,0,Mon,CL1,na,3,CH1,na,1,12,...,Ed Duffy,6.0,pinke101,Ed Pinkham,5.0,zettg101,George Zettlein,1.0,NaN,Y
4,18710509,0,Tue,BS1,na,2,TRO,na,1,9,...,Steve Bellan,5.0,pikel101,Lip Pike,3.0,cravb101,Bill Craver,6.0,HTBF,Y


In [32]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 171907 entries, 0 to 171906
Columns: 161 entries, date to acquisition_info
dtypes: category(78), float64(77), int64(6)
memory usage: 160.5 MB


### merge

In [33]:
df1 = pd.DataFrame({'lkey': ['foo', 'bar', 'baz', 'foo', 'foobar'],
                    'value': [1, 2, 3, 5, 7]})
df2 = pd.DataFrame({'rkey': ['foo', 'bar', 'baz', 'foo', 'foobaz'],
                    'value': [5, 6, 7, 8, 10]})

In [34]:
df1 = df1.set_index('lkey')
df1

,value
lkey,
foo,1
bar,2
baz,3
foo,5
foobar,7


In [35]:
df2 = df2.set_index('rkey')
df2

,value
rkey,
foo,5
bar,6
baz,7
foo,8
foobaz,10


In [36]:
pd.merge(df1, df2, left_index=True, right_index=True)

,value_x,value_y
bar,2,6
baz,3,7
foo,1,5
foo,1,8
foo,5,5
foo,5,8


In [38]:
df1 = df1.reset_index()
df2 = df2.reset_index()

In [39]:
df1.merge(df2, left_on='lkey', right_on='rkey')

,lkey,value_x,rkey,value_y
0,foo,1,foo,5
1,foo,1,foo,8
2,foo,5,foo,5
3,foo,5,foo,8
4,bar,2,bar,6
5,baz,3,baz,7


In [40]:
df1.merge(df2, left_on='lkey', right_on='rkey',
          suffixes=('_left', '_right')) 

,lkey,value_left,rkey,value_right
0,foo,1,foo,5
1,foo,1,foo,8
2,foo,5,foo,5
3,foo,5,foo,8
4,bar,2,bar,6
5,baz,3,baz,7


In [42]:
pd.merge(df1.reset_index(), df2.reset_index(), left_on='lkey', right_on='rkey',
         how='left',
         suffixes=('_left', '_right')) 

,index_left,lkey,value_left,index_right,rkey,value_right
0,0,foo,1,0.0,foo,5.0
1,0,foo,1,3.0,foo,8.0
2,1,bar,2,1.0,bar,6.0
3,2,baz,3,2.0,baz,7.0
4,3,foo,5,0.0,foo,5.0
5,3,foo,5,3.0,foo,8.0
6,4,foobar,7,NaN,NaN,NaN


### resample

In [43]:
df = create_frame(1000000,10000)
df.head()

,name,stamp,value,value2
0,8992,2001-01-01 00:03:07.190,961550,-1.149436
1,9591,2001-01-01 00:08:52.640,127723,-0.538795
2,1464,2001-01-01 00:05:46.408,434488,0.715261
3,6861,2001-01-01 00:06:50.409,81860,-0.619129
4,5074,2001-01-01 00:16:19.350,196685,0.840170


In [44]:
for grouper, grouped_df in df.groupby('name'):
    print(grouper)
    display(grouped_df)
    break

0


,name,stamp,value,value2
14036,0,2001-01-01 00:05:00.236,531447,0.146908
19968,0,2001-01-01 00:02:37.411,985523,-2.372734
21224,0,2001-01-01 00:13:38.319,102725,-0.366287
22173,0,2001-01-01 00:03:31.964,340148,-1.225874
35556,0,2001-01-01 00:05:23.449,717957,0.898235
42290,0,2001-01-01 00:03:49.516,559301,-0.117470
82688,0,2001-01-01 00:04:39.746,244396,-0.768635
88135,0,2001-01-01 00:15:41.661,216179,-0.331614
88838,0,2001-01-01 00:14:16.689,609936,-1.439290
98582,0,2001-01-01 00:05:23.222,649577,-1.558831


[data offsets](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#dateoffset-objects)

In [45]:
df.resample('3min', on='stamp').mean()

,name,value,value2
stamp,,,
2001-01-01 00:00:00,4995.572528,499690.572261,0.003710
2001-01-01 00:03:00,5000.292883,500621.607372,0.004763
2001-01-01 00:06:00,4989.569717,499441.287583,-0.000120
2001-01-01 00:09:00,4999.866672,499748.354422,0.002813
2001-01-01 00:12:00,5009.068100,499829.806206,0.000294
2001-01-01 00:15:00,4997.457490,499270.102420,0.006709


In [46]:
df.set_index('stamp').groupby(['name', pd.Grouper(freq='3min')]).mean().head(15)

value    value2
name stamp                                       
0    2001-01-01 00:00:00  524859.263158  0.062378
     2001-01-01 00:03:00  393250.000000 -0.183712
     2001-01-01 00:06:00  485107.384615  0.127681
     2001-01-01 00:09:00  702491.823529 -0.059155
     2001-01-01 00:12:00  514769.750000 -0.149268
     2001-01-01 00:15:00  543116.384615  0.388144
1    2001-01-01 00:00:00  496185.750000  0.168728
     2001-01-01 00:03:00  450065.416667  0.255239
     2001-01-01 00:06:00  500899.882353  0.157705
     2001-01-01 00:09:00  612660.944444 -0.133513
     2001-01-01 00:12:00  383063.705882 -0.036266
     2001-01-01 00:15:00  427098.714286 -0.031778
2    2001-01-01 00:00:00  440753.937500  0.096411
     2001-01-01 00:03:00  416052.187500  0.174624
     2001-01-01 00:06:00  396571.636364  0.625101

In [47]:
df.groupby('name').resample('3min', on='stamp').mean().head(15)

name          value    value2
name stamp                                             
0    2001-01-01 00:00:00   0.0  524859.263158  0.062378
     2001-01-01 00:03:00   0.0  393250.000000 -0.183712
     2001-01-01 00:06:00   0.0  485107.384615  0.127681
     2001-01-01 00:09:00   0.0  702491.823529 -0.059155
     2001-01-01 00:12:00   0.0  514769.750000 -0.149268
     2001-01-01 00:15:00   0.0  543116.384615  0.388144
1    2001-01-01 00:00:00   1.0  496185.750000  0.168728
     2001-01-01 00:03:00   1.0  450065.416667  0.255239
     2001-01-01 00:06:00   1.0  500899.882353  0.157705
     2001-01-01 00:09:00   1.0  612660.944444 -0.133513
     2001-01-01 00:12:00   1.0  383063.705882 -0.036266
     2001-01-01 00:15:00   1.0  427098.714286 -0.031778
2    2001-01-01 00:00:00   2.0  440753.937500  0.096411
     2001-01-01 00:03:00   2.0  416052.187500  0.174624
     2001-01-01 00:06:00   2.0  396571.636364  0.625101